## Preprocesamiento
1. Conseguir las imágenes, decir origen
2. Reescalarlas
3. Suavizado (explorar con y sin suavizado)
4. Transformar todas las imagenes a dataframe con informacion de cada pixel por columna

Ya tenemos realizado los primeros pasos. Ahora debemos cargar las imagenes para poder armar la base de datos.

In [1]:
import cv2
from skimage import data, color, io
from skimage.transform import rescale, resize, downscale_local_mean
import skimage
from skimage import io

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os

from sklearn.model_selection import train_test_split

import pandas as pd
%matplotlib inline


In [2]:
def suavizar(img):
    """
    Suaviza a la imagen aplicando kernel de convolucion
    """
    n = 2
    kernel = np.ones((n,n),np.float32)*(1./(n**2))
    img_tf = cv2.filter2D(img,-1,kernel)
    return img_tf

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

def load_data(path,w,h):
    """
    Crea la base de datos de las imagenes
    args:
        path: path de la carpeta con imagenes
        w: ancho de imagen
        h: alto de imagen
    returns:
        dataframe con registros de imagenes
    """
    names = os.listdir(path)
    width = w
    height = h
    data = np.zeros([len(names),width*height])
    for i in range(len(names)):
        image = io.imread(path+"/"+names[i], as_gray=True)
        image_resized = resize(image,(height , width),anti_aliasing=True)
        data[i,:] = np.asarray(image_resized).reshape(-1)
    data = pd.DataFrame(data,columns=[str(i) for i in range(width*height)])
    return data

In [3]:
path_con = "C:/Users/juank/Downloads/Fotos_tapabocas_ML/Fotos_tapabocas_ML/con_tapabocas"
path_sin = "C:/Users/juank/Downloads/Fotos_tapabocas_ML/Fotos_tapabocas_ML/sin_tapabocas"
width = 40
height = 80
df_con = load_data(path_con,width,height)
df_sin = load_data(path_sin,width,height)

In [6]:
print(df_con.shape)
print(df_sin.shape)
df = pd.DataFrame(np.concatenate([df_con,df_sin],axis=0), columns = df_con.columns)
df["mask_on"] = np.array(["1"]*df_con.shape[0] + ["0"]*df_sin.shape[0])
df = df.sample(frac=1)

(61, 3200)
(54, 3200)


In [9]:
df["mask_on"] = pd.get_dummies(df["mask_on"],drop_first=True) # 1 si tiene tapabocas, 0 si no lo tiene puesto (o no bien)

In [10]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,3191,3192,3193,3194,3195,3196,3197,3198,3199,mask_on
5,0.777918,0.767929,0.746012,0.719939,0.703129,0.695372,0.694572,0.696028,0.692561,0.684155,...,0.321299,0.328965,0.290175,0.222711,0.236938,0.261646,0.318562,0.361794,0.278664,1
76,0.804689,0.787765,0.757423,0.758491,0.738031,0.732110,0.587861,0.220943,0.165744,0.153348,...,0.093776,0.144923,0.349590,0.591586,0.599921,0.392367,0.100520,0.059394,0.055631,0
54,0.659714,0.669836,0.673495,0.669961,0.674134,0.681978,0.687247,0.688851,0.690676,0.693165,...,0.223025,0.235705,0.294241,0.296647,0.281040,0.277377,0.217485,0.189459,0.242043,1
26,0.240938,0.237144,0.269894,0.236344,0.237556,0.243640,0.259936,0.268911,0.254092,0.247310,...,0.835882,0.815235,0.800276,0.848811,0.816162,0.810933,0.788310,0.671688,0.790662,1
109,0.225055,0.318353,0.510806,0.543406,0.556500,0.560271,0.414889,0.143324,0.111081,0.105434,...,0.424345,0.395795,0.387297,0.384792,0.384307,0.393123,0.390811,0.374581,0.374923,0


----

## Implementación de redes neuronales
### Separación de datos de entrenamiento y de prueba
Ayuda: https://keras.io/api/optimizers/

In [11]:
X = df.iloc[:,:-1]
y = df["mask_on"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [12]:
print(len("Train size: ",X_train))
print(len("Test size: ", X_test))



92
23


In [13]:
width*height

3200

In [14]:
# Inicializando los parametros de la red neuronal
import keras
from keras import models
from keras import layers

In [20]:
network = models.Sequential()
network.add(layers.Dense(4000, activation='sigmoid', input_shape=(width*height,)))
network.add(layers.Dense(7000,activation='relu'))
network.add(layers.Dense(5000,activation='relu'))
network.add(layers.Dense(3000,activation='relu'))
network.add(layers.Dense(2000,activation='relu'))
network.add(layers.Dense(1000,activation='relu'))
network.add(layers.Dense(100,activation='relu'))

#network.add(layers.Dense(5000,activation='relu'))
#network.add(layers.Dense(7000,activation='relu'))
#network.add(layers.Dense(5000,activation='relu'))
#network.add(layers.Dense(7000,activation='relu'))

network.add(layers.Dense(1,activation='relu'))
#network.add(layers.Dense(3000,activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=1e-15)
#model.compile(loss='categorical_crossentropy', optimizer=opt)

network.compile(optimizer = opt,
                loss='categorical_crossentropy',
               metrics=['accuracy'])

In [21]:
network.fit(X_train, y_train, epochs=10,batch_size=5)

Epoch 1/10
19/19 [==============================] - 14s 761ms/step - loss: nan - accuracy: 0.4565
Epoch 2/10
19/19 [==============================] - 14s 740ms/step - loss: nan - accuracy: 0.4565
Epoch 3/10
19/19 [==============================] - 15s 775ms/step - loss: nan - accuracy: 0.4565
Epoch 4/10
19/19 [==============================] - 14s 732ms/step - loss: nan - accuracy: 0.4565
Epoch 5/10
19/19 [==============================] - 14s 729ms/step - loss: nan - accuracy: 0.4565
Epoch 6/10
19/19 [==============================] - 14s 745ms/step - loss: nan - accuracy: 0.4565
Epoch 7/10
19/19 [==============================] - 14s 744ms/step - loss: nan - accuracy: 0.4565
Epoch 8/10
19/19 [==============================] - 14s 744ms/step - loss: nan - accuracy: 0.4565
Epoch 9/10
19/19 [==============================] - 14s 740ms/step - loss: nan - accuracy: 0.4565
Epoch 10/10
19/19 [==============================] - 14s 758ms/step - loss: nan - accuracy: 0.4565
